In [1]:
import socket
import sys
import requests
import requests_oauthlib
import json

In [2]:
CONSUMER_KEY = 'your twitter consumer key'
CONSUMER_SECRET = 'your twitter consumer secret'
ACCESS_TOKEN = 'your twitter access token'
ACCESS_SECRET = 'your twitter access secret'
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_SECRET)

In this example, it searches and streams tweets with keywoard Python

In [5]:
def download_matched_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    parameters = [('language', 'en'),('track','Python')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in parameters])
    response = requests.get(query_url, auth=my_auth, stream=True)
    return response

response.iter_lines() is an endless iterator, in this example, it will be stopped by KeyboardInterrupt exception

In [6]:
def send_over_socket(http_resp, connection):
    try:
        for line in http_resp.iter_lines():
            try:
                full_tweet = json.loads(line)
                tweet_text = full_tweet['text']
                print("Tweet Text: " + tweet_text)
                print ("------------------------------------------")
                tweet=tweet_text+'\n'
#send over tcp socket requires byte object instead of string, therefore use encode() to convert to bytes
                connection.send(tweet.encode())
            except:    
                err = sys.exc_info()[0]
                print("Error: %s" % err)
    except KeyboardInterrupt:
        pass   

In [7]:
TCP_IP = "10.0.0.46"
TCP_PORT = 20000
conn = None
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind((TCP_IP, TCP_PORT))
sock.listen(1)
print("Waiting for TCP connection...")
conn, addr = sock.accept()
print("Connected...")
resp = download_matched_tweets()
send_over_socket(resp, conn)

Waiting for TCP connection...
Connected...
Error: <class 'json.decoder.JSONDecodeError'>
Tweet Text: @grantdmckenzie And ultimately you know best for your students needs! For semester long course, 2 weeks is not that… https://t.co/zzSjVrJn2f
------------------------------------------
Tweet Text: Checkout CursorTek is #selling #bulk #data of 3k plus #categories like:

#bars #hotels #restaurants #plumbers… https://t.co/l3sKNmjzJq
------------------------------------------
Tweet Text: RT @DynamicDuda338: Sooo... today... I wasn’t feeling too good. 😵 I decided to stay in my PJs and complete 8 short classes of Python coding…
------------------------------------------
Tweet Text: RT @pauli_rearea: @grantdmckenzie And ultimately you know best for your students needs! For semester long course, 2 weeks is not that much…
------------------------------------------
